In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [1]:
from typing import Annotated
from typing_extensions import TypedDict

from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model="gpt-4o")




In [23]:
from pydantic import BaseModel, Field
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from enum import Enum

class Decision(str, Enum):
    CONTINUE = "CONTINUAR"
    ESCALAR = "ESCALAR"
    NO_REMINDERS = "NO REMINDERS"

class ResponseFormatter(BaseModel):
        """Always use this tool to structure your response to the user."""

        decision: Decision  = Field(description="The decision to make")
        reason: str = Field(description="The reason for the decision")
        summary: str = Field(description="A summary of the conversation")

system_prompt = SystemMessage(
        content="""
Eres un analizador de conversaciones especializado en evaluar interacciones entre usuarios y un bot de atención al cliente. Tu tarea es examinar la interacción y determinar el mejor curso de acción.

El bot tiene las siguientes capacidades:
- Onboarding y recolección de información inicial
- Perfilado de usuarios en diferentes niveles (L0/L1 y L2)
- Programación de citas y seguimiento
- Enrutamiento inteligente de conversaciones
- Síntesis y resumen de información

Debes decidir entre tres opciones:

1. CONTINUAR (OPCIÓN POR DEFECTO):
   - La conversación fluye normalmente
   - El bot puede manejar la consulta
   - El usuario está satisfecho con las respuestas
   - La información solicitada está dentro del alcance del bot
   - El bot está proporcionando información útil y relevante

2. ESCALAR (SOLO en estos casos específicos):
   - El usuario EXPLÍCITAMENTE dice estar molesto o frustrado (ej: "estoy harto", "no me estás ayudando")
   - El bot EXPLÍCITAMENTE indica que no puede resolver la solicitud dos veces, solo en caso de que suceda tres veces (ej: "lo siento, no puedo ayudarte con eso")
   - El usuario EXPLÍCITAMENTE solicita hablar con un humano (ej: "quiero hablar con una persona")
   - El usuario muestra enojo EXPLÍCITO en más de 2 mensajes consecutivos

3. NO REMINDERS (SOLO en estos casos específicos):
   - Se han enviado 2 o más recordatorios consecutivos sin respuesta
   - La conversación ha sido marcada como finalizada
   - El usuario ha indicado explícitamente que no desea más mensajes
   - Se detecta un patrón de mensajes repetitivos del bot

IMPORTANTE:
- Por defecto, SIEMPRE elige CONTINUAR a menos que se cumplan EXACTAMENTE los criterios para ESCALAR o NO REMINDERS
- NO interpretes o infierras situaciones - usa SOLO los criterios explícitos mencionados
- La decisión debe basarse en hechos concretos, no en suposiciones
- NO escales por complejidad de la consulta a menos que el bot EXPLÍCITAMENTE diga que no puede resolverla
- NO escales por precios o detalles específicos a menos que el bot EXPLÍCITAMENTE diga que no puede proporcionarlos

Devuelve SOLO una de estas tres palabras: CONTINUAR, ESCALAR, NO REMINDERS.
Tu decisión debe basarse en un análisis completo del contexto y la interacción.
    """
    )

model = llm.with_structured_output(ResponseFormatter)

In [25]:
res =model.invoke([
    system_prompt,
    HumanMessage(content="Hola, estoy buscando un espacio para mi empresa"),
    AIMessage(
        content="Hola, soy Sofía de Spot2 😊. Para ayudarte mejor, ¿qué tipo de espacio estás buscando: local comercial, oficina o bodega industrial?"
        ),
        HumanMessage(content="rentar una oficina para vender bebidas"),
        HumanMessage(
            content="1000m2 3000mxn total de la propiedad en polanco",
        ),
        AIMessage(
            content="Lo siento, no puedo ayudarte con eso. ¿Puedo ayudarte con algo más?"
        ),
        HumanMessage(content="Por favor, habla con un humano"),
])

print(res.model_dump_json())

{"decision":"ESCALAR","reason":"El usuario ha solicitado explícitamente hablar con un humano.","summary":"El usuario está buscando rentar una oficina para vender bebidas y pide que el total sea 3000 MXN en Polanco. Ha solicitado hablar con un humano explícitamente despues de que el bot indicó que no puede ayudar con eso."}


In [27]:
res =model.invoke([
    system_prompt,
    HumanMessage(content="Hola, estoy buscando un espacio para mi empresa"),
    AIMessage(
        content="Hola, soy Sofía de Spot2 😊. Para ayudarte mejor, ¿qué tipo de espacio estás buscando: local comercial, oficina o bodega industrial?"
        ),
        HumanMessage(content="rentar una oficina para vender bebidas"),
        HumanMessage(
            content="1000m2 3000mxn total de la propiedad en polanco",
        ),
        AIMessage(
            content="Lo siento, no puedo ayudarte con eso. ¿Puedo ayudarte con algo más?"
        ),
        AIMessage(content="Hola, sigues ahi?"),
        AIMessage(content="Hola, mira este es tu proyecto, quieres continuar?"),
])

print(res.model_dump_json())

{"decision":"NO REMINDERS","reason":"Se han enviado 2 recordatorios consecutivos sin respuesta del usuario.","summary":"El usuario solicita información para rentar una oficina en Polanco pero después no responde a los recordatorios enviados por el bot."}
